In [15]:
import pandas as pd
import numpy as np
import re
import os
import import_ipynb
from modes_prepare_lib import get_modes_dataframe
from datetime import datetime
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

In [16]:
df_modes, df_valid = get_modes_dataframe()

129559
(129086, 38)
4529
(4507, 24)


In [17]:
df_modes.columns, df_valid.columns

(Index(['№ пакета', '№ партии', 'НД (гост, ту)', 'марка стали',
        'Дата термообработки', '1 зона по ВТР закалка',
        '1 зона уставка закалка', '1 зона фактическая температура закалка',
        '2 зона по ВТР закалка', '3 зона по ВТР закалка',
        '3 зона уставка закалка', '3 зона фактическая температура закалка',
        'Температура трубы на выгрузке из печи закалки',
        'Скорость прохождения трубы через спрейер, м/с',
        't˚ C трубы после спреера', 'Расход воды на закалочный спрейер, м3',
        'время выдержки в закалочной ванне, сек.',
        't˚ C воды в закалочной ванне', 't˚ C трубы после ванны',
        '1 зона ВТР и уставка отпуск', '1 зона фактическая температура отпуск',
        '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
        '3 зона фактическая температура отпуск', '4 зона ВТР и уставка отпуск',
        '5 зона ВТР и уставка отпуск', '5 зона фактическая температура отпуск',
        'Температура трубы на выгрузке из печи отпус

In [18]:
def change_to17(x):
    if x.year>2018:
        try:
            tmp=x
            tmp = tmp.replace(year=2017, month=x.month, day=x.day)
            return tmp
        except:
            tmp = tmp.replace(year=2017, month=x.month, day=28)
            return tmp
    else:
        return x

def union_modes():
    global df_modes
    global df_valid
    result=list(set(df_modes.columns) & set(df_valid.columns))
    df_modes = df_modes[result]
    df_modes['Дата термообработки'].describe()
    df_valid = df_valid[result]
    
    df = pd.concat([df_modes, df_valid])
    df.reset_index(drop=True, inplace=True)
    return df

df = union_modes()

In [19]:
def transform_date(df):
    """
    Приводим дату к общему формату
    Все к datetime
    """
    timestamp_df = df[df['Дата термообработки'].apply(type)==pd._libs.tslib.Timestamp]
    last_df = df[df['Дата термообработки'].apply(type)!=pd._libs.tslib.Timestamp]
    last_df['Дата термообработки'] = last_df['Дата термообработки'].apply(
        lambda x:datetime.strptime(x, "%d.%m.%Y"))
    timestamp_df['Дата термообработки'] = timestamp_df['Дата термообработки'].apply(
        pd._libs.tslib.Timestamp.to_pydatetime)
    last_df['Дата термообработки'] = last_df['Дата термообработки'].apply(change_to17)
    print(last_df[last_df['Дата термообработки'].apply(lambda x:x.year)>2018].shape)
    last_df.shape
    df = pd.concat([last_df, timestamp_df])
    df.reset_index(drop=True, inplace=True)
    return df

In [20]:
df = transform_date(df)

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Repo\Anaconda2\envs\myenv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

(0, 22)


In [21]:
df.columns

Index(['Скорость прохождения трубы через спрейер, м/с',
       '4 зона ВТР и уставка отпуск', 'Дата термообработки',
       '5 зона ВТР и уставка отпуск', 'марка стали', 'толщина стенки',
       '3 зона по ВТР закалка', '3 зона ВТР и уставка отпуск',
       '1 зона по ВТР закалка', '№ пакета', '№ партии',
       '1 зона ВТР и уставка отпуск', 'диаметр', '№ плавки',
       'время выдержки в закалочной ванне, сек.',
       'шаг балок закалочная печь, сек', '2 зона по ВТР закалка',
       't˚ C трубы после спреера', '2 зона ВТР и уставка отпуск',
       't˚ C трубы после ванны', 'шаг балок отпускная печь, сек',
       'НД (гост, ту)'],
      dtype='object')

In [22]:
def isDigit(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

In [23]:
df = df[df['диаметр'].apply(isDigit)]
df = df[df['толщина стенки'].apply(isDigit)]
df.shape

(130595, 22)

In [24]:
# df['диаметр'].value_counts().sort_index()

In [25]:
df['диаметр'] = df['диаметр'].astype(float)
df['толщина стенки'] = df['толщина стенки'].astype(float)

In [47]:
def find_mark_typesize(mark, d, s):
    answ = df.copy()
    if mark!=None:
        answ = answ[answ['марка стали']==mark]
#     print(answ.shape)
    if d!=None:
        answ = answ[answ['диаметр']==d]
#     print(answ.shape)
    if s!=None:
        answ = answ[answ['толщина стенки']==s]
#     print(answ.shape)
    return answ

In [86]:
# 127х12,5 м/ст. 37г2ф, гост Р53366, гр.пр. к72

answ = find_mark_typesize('37Г2Ф', 127, 12.5)
answ.shape


(84, 22)

In [81]:
ls_ = ['№ партии', 'марка стали', '1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера',
       'время выдержки в закалочной ванне, сек.', 't˚ C трубы после ванны',
       '1 зона ВТР и уставка отпуск', '2 зона ВТР и уставка отпуск',
       '3 зона ВТР и уставка отпуск', '4 зона ВТР и уставка отпуск',
       '5 зона ВТР и уставка отпуск', 'шаг балок закалочная печь, сек',
       'шаг балок отпускная печь, сек', 'диаметр', 'толщина стенки', 'Дата термообработки']

In [83]:
answ = answ[ls_]
answ.sort_values('Дата термообработки', ascending=False).to_excel('../DATA/old_modes_to_saw/127x12,5 37g2f.xlsx')

In [84]:
# answ['марка стали'].value_counts()

In [87]:
answ['толщина стенки'].value_counts()

12.5    84
Name: толщина стенки, dtype: int64

In [88]:
answ

,марка стали,2 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,2 зона по ВТР закалка,t˚ C трубы после спреера,№ партии,толщина стенки,"шаг балок отпускная печь, сек",№ плавки,"НД (гост, ту)",1 зона по ВТР закалка,1 зона ВТР и уставка отпуск,№ пакета,"время выдержки в закалочной ванне, сек.",3 зона ВТР и уставка отпуск,3 зона по ВТР закалка,"шаг балок закалочная печь, сек",t˚ C трубы после ванны,диаметр,Дата термообработки,"Скорость прохождения трубы через спрейер, м/с",4 зона ВТР и уставка отпуск
231,37Г2Ф,615,643,840,42.5,701083,12.5,36,6f4085,Г 632-80,840,671,402733878,,660,850,36,NaN,127.0,2017-01-04,0.13,607
232,37Г2Ф,615,618,840,42.5,701084,12.5,36,6f5866,Г 632-80,840,644,402733894,,634,850,36,NaN,127.0,2017-01-04,0.13,607
233,37Г2Ф,615,599,840,42.5,701084,12.5,36,6f5866,Г 632-80,840,623,402733911,,615,850,36,NaN,127.0,2017-01-04,0.13,607
1568,37Г2Ф,615,633,840,65.0,701364,12.5,36,6f7983,ТУ 14,840,660,75161243,,649,850,36,NaN,127.0,2017-01-26,0.18,607
1569,37Г2Ф,615,633,840,65.0,701364,12.5,36,6f7983,ТУ 14,840,660,75161222,,649,850,36,NaN,127.0,2017-01-26,0.18,607
1570,37Г2Ф,615,633,840,65.0,701364,12.5,36,6f7983,ТУ 14,840,640,75161258,,649,850,36,NaN,127.0,2017-01-26,0.18,607
1571,37Г2Ф,615,633,840,65.0,701364,12.5,36,6f7983,ТУ 14,840,640,75161246,,649,850,36,NaN,127.0,2017-01-26,0.18,607
4509,37Г2Ф,670,661,840,72.5,703204,12.5,36,7f0873,ТУ 14,840,690,75180235,,677,850,40,NaN,127.0,2017-03-16,0.15,659
4510,37Г2Ф,670,661,840,72.5,703204,12.5,36,7f0873,ТУ 14,840,690,75180460,,677,850,40,NaN,127.0,2017-03-16,0.15,659
4511,37Г2Ф,670,661,840,72.5,703204,12.5,36,7f0873,ТУ 14,840,690,75179992,,677,850,40,NaN,127.0,2017-03-16,0.15,659


In [14]:
plan_may = pd.read_excel('D:\\Repo\\PycharmProjects\\gitPNTZ\\tmp_files\\план на май\\план_на_май_ПНТЗ.xlsx')
plan_may = plan_may[['Сортамент|Марка стали','Сортамент|Диам1', 'Сортамент|Стен1']]
plan_may.columns = [['марка стали', 'диаметр', 'толщина стенки']]
plan_may.drop_duplicates(inplace=True)
plan_may.reset_index(drop=True, inplace=True)

In [15]:
def close_value(df, row, value):
    s_close = df[row].value_counts().index[0]
    min_delta = np.abs(s_close-value)
    for s_ in df[row].value_counts().index:
        delta = np.abs(s_-value)
        if delta<min_delta:
            min_delta = delta
            s_close = s_
    return s_close

In [18]:
def find_close_sort(mark, d, s):
    answ = df.copy()
    if answ[answ['марка стали']==d].shape[0]==0:
        mark = mark.split('-')[0]
    answ = answ[answ['марка стали']==mark]
    if answ[answ['диаметр']==d].shape[0]!=0:
        answ = answ[answ['диаметр']==d]
        s_close = close_value(answ, 'толщина стенки',s)
        answ = answ[answ['толщина стенки']==s_close]
    elif answ[answ['толщина стенки']==s].shape[0]!=0:
        answ = answ[answ['толщина стенки']==s]
        d_close = close_value(answ, 'диаметр',d)
#         print(answ.shape)
        answ = answ[answ['диаметр']==d_close]
    return answ 

In [19]:
modes_old = pd.DataFrame()

for i in plan_may.index:
    mark = plan_may.iloc[i, 0]
    d = plan_may.iloc[i,1]
    s = plan_may.iloc[i,2]
    s = s.round(2)
#     print(mark, d, s)
    tmp = find_mark_typesize(mark, d, s)
    if tmp.shape[0]!=0:
        modes_old = pd.concat([modes_old, tmp.sort_values('Дата термообработки', ascending=False).head(1)])
    else:
        tmp = pd.DataFrame({
            '0_марка стали': [mark],
            '0_диаметр':[d],
            '0_толщина стенки':[s]})
        tmp_right = find_close_sort(mark, d, s)
        st = tmp_right.sort_values('Дата термообработки', ascending=False).head(1)
        st.reset_index(drop=True, inplace=True)
        tmp = pd.concat([tmp,st], axis=1)
        modes_old = pd.concat([modes_old, tmp])
#     print(mark, d, s)

In [22]:
modes_old

,0_диаметр,0_марка стали,0_толщина стенки,1 зона ВТР и уставка отпуск,1 зона по ВТР закалка,2 зона ВТР и уставка отпуск,2 зона по ВТР закалка,3 зона ВТР и уставка отпуск,3 зона по ВТР закалка,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,t˚ C трубы после ванны,t˚ C трубы после спреера,Дата термообработки,"НД (гост, ту)","Скорость прохождения трубы через спрейер, м/с","время выдержки в закалочной ванне, сек.",диаметр,марка стали,толщина стенки,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",№ пакета,№ партии,№ плавки
126204,NaN,NaN,NaN,630,840,630,840,621,850,621,606,NaN,95.0,2018-02-25,ТУ 21,0.7,,73.02,37Г2Ф,5.51,24,24,403624946,802343,8f1228
0,146.00,37Г2Ф,7.70,602,840,655,840,594,850,644,582,NaN,42.5,2016-08-28,ТУ 13,0.25,,146.00,37Г2Ф,9.50,36,36,402616866,608301,6f5291
126116,NaN,NaN,NaN,645,850,645,850,635,860,635,619,NaN,80.0,2018-02-23,ТУ 9,0.7,,73.02,30Г2,5.51,24,24,403611036,802328,8f0950
46904,NaN,NaN,NaN,675,850,675,850,663,860,663,647,NaN,47.5,2013-07-28,ТУ 21,0.67,,60.32,38Г2С,5.00,27,27,74139653,307295,3f3714
82920,NaN,NaN,NaN,586,850,520,850,579,860,517,567,NaN,45.0,2015-05-16,8732/8731,0.3,,114.00,30Г2,8.60,33,33,7537,505258,5f2014
0,168.28,30Г2,7.32,605,850,605,850,597,860,597,584,NaN,75.0,2018-02-27,8732/31,0.32,,168.28,30Г2,8.00,30,30,4562,802369,8f0862
13205,NaN,NaN,NaN,675,850,675,850,663,860,663,647,NaN,70.0,2017-09-12,ТУ 21,0.7,,60.32,30Г2,5.00,24,24,75359794,709183,7f5961
78322,NaN,NaN,NaN,620,850,620,850,611,860,611,597,NaN,55.0,2015-03-14,ТУ 21,0.45,,88.90,30Г2,7.34,30,27,1.01131e+08,503201,511151
0,178.00,37Г2Ф,8.10,525,830,525,830,522,840,522,521,NaN,85.0,2018-02-09,8732/31,0.27,,178.00,37Г2Ф,9.20,30,30,3809,802100,8f0692
126473,NaN,NaN,NaN,600,850,600,850,593,866,593,580,NaN,75.0,2018-02-28,8732/31,0.37,,168.28,30Г2,8.94,30,30,5704,802385,8f0971


In [23]:
modes_old.to_excel('modes_old.xlsx')